In [72]:
import numpy as np
import os
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adadelta
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle

In [21]:
# Set seed
SEED = 1234
np.random.seed(SEED)

In [101]:
INPUT_SHAPE = (132, 1300, 1)
BATCH_SIZE = 32
MODEL_SAVE_NAME = "autoencoder.model"
# FEATURE_DIR = "/home/ubuntu/music_influence/data/features/mel_spec_first"
FEATURE_DIR = "../data/features/mel_spec_first/"

In [85]:
# Train-val split
all_filenames = os.listdir(FEATURE_DIR)
train, val = train_test_split(all_filenames, test_size=0.25, random_state=SEED)

In [110]:
np.save('autoencoder_split/train_filenames.npy', np.array(train))
np.save('autoencoder_split/val_filenames.npy', np.array(val))

In [107]:
# Architecture adapted from 
# https://blog.keras.io/building-autoencoders-in-keras.html and
input_img = Input(INPUT_SHAPE)

# Encoder
encoded = Conv2D(64, (3, 3), input_shape=INPUT_SHAPE, activation='relu', padding='same')(input_img)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
encoded = Conv2D(32, (3, 3), input_shape=INPUT_SHAPE, activation='relu', padding='same')(encoded)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
encoded = Conv2D(16, (3, 3), input_shape=INPUT_SHAPE, activation='relu', padding='same')(encoded)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
encoder = Model(input=input_img, output=encoded)

# Decoder
decoded = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(32, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(64, (3, 3), activation='relu')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
# TODO:
decoded = Conv2D(1, (3, 3), activation='relu', padding='same')(decoded)

# Create learning rate schedule and add it to the optimizer of choice
learning_rate = 1.0
epochs = 50
decay_rate = learning_rate / epochs
adadelta = Adadelta(lr=learning_rate, rho=0.95, epsilon=1e-08, decay=decay_rate)

# Create a custom callback to stop training the autoencoder when a val_loss of 0.1 is reached
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.1, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

callbacks = [
    EarlyStoppingByLossVal(monitor='val_loss', value=0.1, verbose=1),
    ModelCheckpoint(MODEL_SAVE_NAME, monitor='val_loss', save_best_only=True, verbose=0),
]

# Build the full autoencoder
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=adadelta, loss='binary_crossentropy')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 132, 1300, 1)      0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 132, 1300, 64)     640       
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 66, 650, 64)       0         
_________________________________________________________________
conv2d_128 (Conv2D)          (None, 66, 650, 32)       18464     
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 33, 325, 32)       0         
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 33, 325, 16)       4624      
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 17, 163, 16)       0         
__________

/Users/harryxue/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ma..., inputs=Tensor("in...)`
  if sys.path[0] == '':


In [95]:
def generator(paths, batch_size=BATCH_SIZE):
    """
        Infinite generator for batches of data
    """
    batch = []
    
    while True:            
        for path in paths:
            mel_spec = np.load(FEATURE_DIR + path)
            # Add zero padding
            mel_spec_padded = np.zeros(INPUT_SHAPE[:2])
            mel_spec_padded[:mel_spec.shape[0], :mel_spec.shape[1]] = mel_spec
            batch.append(mel_spec_padded[:, :, np.newaxis])
            
            if len(batch) == batch_size:
                yield np.array(batch), np.array(batch)
                batch = []

In [104]:
history = autoencoder.fit_generator(
                    generator = generator(train),
                    epochs = 50,
                    steps_per_epoch = len(train)//BATCH_SIZE,
                    validation_data = generator(val),
                    validation_steps = len(val)//BATCH_SIZE,
                    callbacks=callbacks,
                    shuffle=True,
                    verbose=1
            )

Epoch 1/50


KeyboardInterrupt: 

In [ ]:
# Save history callback object
with open('history.pickle', 'wb') as handle:
    pickle.dump(history.history, handle)